# Создание и заполнение данных БД Postgre

In [1]:
%pip install python-dotenv psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 1.4 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.7 MB 1.4 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.7 MB 1.3 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 1.2 MB/s eta 0:00:02
   ----------------------- ---------------- 1.6/2.7 MB 1.3 MB/s eta 0:00:01
   --------------------------- ------------ 1.8/2.7 MB 1.2 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.7 MB 1.2 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.7 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart t


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\maxpl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor
from dotenv import load_dotenv


# Получение секретов

In [6]:
# получаем текущую директорию ноутбука 
current_dir = os.getcwd()

# переходим на один уровень вверх
project_root = os.path.dirname(current_dir)

# формируем путь к файлу .env в папке Task1, там у нас лежит файл .env с настройками подключения к БД
dotenv_path = os.path.join(project_root, 'task2_Docker', '.env')

# загружаем переменные окружения из указанного файла
load_dotenv(dotenv_path)

# получим доступ к переменным окружения
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")
db_port = os.getenv("DB_PORT") 
secret_hash = os.getenv("SECRET_HASH")

print(f"Загруженные данные: USER={user}, DB={db_name}, DB_PORT={db_port}")

Загруженные данные: USER=Maxim, DB=Plusnin, DB_PORT=5433


# Подключение к базе данных PostgreSQL

In [13]:
conn = None
try:
    conn = psycopg2.connect(
        host="localhost", # если Docker контейнер запущен локально, а ноутбук вне Docker.
                          # НО! если ноутбук также в Docker и в одной сети с БД,
                          # то нужно использовать имя сервиса Docker (например, 'db' или 'postgres_db').
        database=db_name,
        user=user,
        password=password,
        port=db_port
    )
    cursor = conn.cursor()

    print("Успешное подключение к базе данных!")
    
except Exception as e:
    print(f"Ошибка при подключении к базе данных: {e}")

Успешное подключение к базе данных!


In [9]:
# пример запроса
cursor.execute("SELECT version();")
db_version = cursor.fetchone()
print(f"Версия PostgreSQL: {db_version}")

Версия PostgreSQL: ('PostgreSQL 13.23 (Debian 13.23-1.pgdg13+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit',)


In [10]:
# получить список таблиц:
cursor.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
""")
tables = cursor.fetchall()
print("\nТаблицы в базе данных:")
for table in tables:
    print(f"- {table[0]}")


Таблицы в базе данных:
- departments
- user_logs


In [17]:
# закрытие соединения с БД - После завершения работы с БД не забываем закрывать соединение!
cursor.close()
conn.close()

Вам предоставлена БД с логами (действиями) студентов на образовательном портале за весенний семестр (агрегация по каждой неделе) по отдельному электронному курсу - таблица user_logs (примечание. создана в предыдущих л.р.).
- сourseid — уникальный идентификатор курса, дисциплины;
- userid — уникальный идентификатор студента (не используется в обучении);
- num_week — номер недели в году;
- s_all — количество всех событий на текущий момент;
- s_all_avg — среднее количество всех событий в неделю;
- s_course_viewed — количество просмотров курса;
- s_course_viewed_avg — среднее количество просмотров курса в неделю;
- s_q_attempt_viewed — количество просмотров теста;
- s_q_attempt_viewed_avg — среднее количество просмотров теста в неделю;
- s_a_course_module_viewed — количество просмотров модуля в курсе;
- s_a_course_module_viewed_avg — среднее количество просмотров модуля в курсе в неделю;
- s_a_submission_status_viewed — количество отправленных заданий на проверку;
- s_a_submission_status_viewed_avg — среднее количество ответов;
- namer_level — оценка за дисциплину;
- depart — номер кафедры;
- name_osno — основа обучения (имеет два значения: бюджет или контракт);
- name_formopril — форма обучения;
- leveled — уровень образования (имеет два значения: бакалавриат, магистратура);
- num_sem — номер семестра;
- kurs — номер курса учебной группы.

Также в таблице  departments хранятся названия кафедр, таблица связана с логами по полю depart:
id - код кафедры;
name - сокращенное название кафедры. 

In [5]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor
from dotenv import load_dotenv
# получаем текущую директорию ноутбука 
current_dir = os.getcwd()

# переходим на один уровень вверх
project_root = os.path.dirname(current_dir)

# формируем путь к файлу .env в папке Task1, там у нас лежит файл .env с настройками подключения к БД
dotenv_path = os.path.join(project_root, 'task3_Docker', '.env')

# загружаем переменные окружения из указанного файла
load_dotenv(dotenv_path)

# получим доступ к переменным окружения
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")
db_port = os.getenv("DB_PORT") 
secret_hash = os.getenv("SECRET_HASH") 

print(f"Загруженные данные: USER={user}, DB={db_name}, DB_PORT={db_port}")
conn = None
try:
    conn = psycopg2.connect(
        host="localhost", # если Docker контейнер запущен локально, а ноутбук вне Docker.
                          # НО! если ноутбук также в Docker и в одной сети с БД,
                          # то нужно использовать имя сервиса Docker (например, 'db' или 'postgres_db').
        database=db_name,
        user=user,
        password=password,
        port=db_port
    )
    cursor = conn.cursor()

    print("Успешное подключение к базе данных!")
    
except Exception as e:
    print(f"Ошибка при подключении к базе данных: {e}")

Загруженные данные: USER=Maxim, DB=Plusnin, DB_PORT=5433
Успешное подключение к базе данных!


## Задание 1 (если до этого еще этот шаг не был выполнен):

Измените данные вещественного типа, сейчас целая и дробная часть разделены запятой, замените ее на точку. 

Выведите первые 10 записей, чтобы проверить результат предобработки. 

In [25]:
# Заменяем запятые на точки в текстовых полях, хранящих вещественные числа
cursor.execute("""
UPDATE user_logs
SET s_all_avg = REPLACE(s_all_avg, ',', '.'),
    s_course_viewed_avg = REPLACE(s_course_viewed_avg, ',', '.'),
    s_q_attempt_viewed_avg = REPLACE(s_q_attempt_viewed_avg, ',', '.'),
    s_a_course_module_viewed_avg = REPLACE(s_a_course_module_viewed_avg, ',', '.'),
    s_a_submission_status_viewed_avg = REPLACE(s_a_submission_status_viewed_avg, ',', '.');
""")
conn.commit()


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
conn.rollback() #Убираем прошлую транзакцию, потому что она была с ошибкой (Данные уже были преведены в числовой формат)

In [6]:
cursor.execute('SELECT * FROM user_logs LIMIT 10;')
rows = cursor.fetchall()

print("Первые 10 записей:")
for row in rows:
    print(row)

Первые 10 записей:
(71262, 34527, 6, 9, 9.0, 4, 4.0, 0, 0.0, 0, 0.0, 0, 0.0, '3', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34609, 6, 6, 6.0, 3, 3.0, 0, 0.0, 0, 0.0, 0, 0.0, '2', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34610, 6, 13, 13.0, 5, 5.0, 0, 0.0, 1, 1.0, 1, 1.0, '5', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34611, 6, 12, 12.0, 7, 7.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экзамен', 22, '2', '1', '1', 2, 2, '18.06.2022')
(71262, 34612, 6, 24, 24.0, 8, 8.0, 0, 0.0, 0, 0.0, 0, 0.0, '3', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34613, 6, 15, 15.0, 8, 8.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34615, 6, 14, 14.0, 7, 7.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34616, 6, 5, 5.0, 2, 2.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экзамен', 22, '1', '1', '1', 2, 2, '18.06.2022')
(71262, 34616, 8, 2, 3.0, 2, 2.0, 0, 0.0, 0, 0.0, 0, 0.0, '4', 'Экз

## Задание 2: 

Выведите количество кафедр, за которыми закреплены курсы на портале.





In [31]:
conn.rollback() 

In [48]:
cursor.execute('''SELECT COUNT(DISTINCT depart)
               FROM user_logs;''')
count = cursor.fetchone()[0]
print(f"Количество кафедр, за которыми закреплены курсы: {count}")

Количество кафедр, за которыми закреплены курсы: 43


##  Задание 3:

Выведите сколько у каждой кафедры закреплено электронных курсов на портале. 
Требуется выводит сокращенное название кафедры и количество курсов. 
У какой кафедры больше всего курсов на портале?

In [7]:
cursor.execute("""
SELECT d.name, COUNT(DISTINCT u.courseid) as course_count
FROM user_logs u
JOIN departments d ON u.depart = d.id
GROUP BY d.name;
""")
rows = cursor.fetchall()
print("Кафедра - Количество курсов")
for row in rows:
    print(f"{row[0]} - {row[1]}")

def last_znch (x):
    return x[1]
print(max(rows, key=last_znch))

Кафедра - Количество курсов
CC - 19
АиИИ - 19
АСУ - 16
АЭПиМ - 33
БИиИТ - 35
ВИ - 22
ВТиП - 25
ГМДиОПИ - 29
ГМиТТК - 40
ГМУиУП - 35
Дизайна - 20
ДиСО - 53
ИиИБ - 16
ИТМ - 3
ЛиП - 33
ЛиУТС - 36
ЛПиМ - 28
Менеджм. - 20
МиТОДиМ - 28
МиХТ - 42
ПиСЗ - 32
ПиЭММО - 33
ПМиИ - 19
ПОиД - 42
Психол. - 23
ПЭиБЖД - 11
РМПИ - 28
РЯОЯиМК - 17
СРиППО - 14
ТиЭС - 19
ТОМ - 30
ТССА - 21
УиИС - 15
УСиБА - 2
Физики - 16
Физкульт. - 5
Химии - 18
ХОМ - 18
ЦДОМ - 4
ЭиМЭ - 16
Эконом. - 32
ЭПП - 16
ЯиЛ - 20
('ДиСО', 53)


## Задание 4:

Ответьте на вопрос: существуют ли курсы, за которыми закреплено несколько кафедр? Если такие курсы есть, то выведите их количество.
Также выведите названия кафедр, которые совместно преподают один и тот же курс.




In [11]:
# Ищем курсы с >1 кафедрой
cursor.execute("""
SELECT u.courseid, STRING_AGG(DISTINCT d.name, ', ') as departments
FROM user_logs u
JOIN departments d ON u.depart = d.id
WHERE u.courseid IN (
    SELECT courseid 
    FROM user_logs 
    GROUP BY courseid 
    HAVING COUNT(DISTINCT depart) > 1
)
GROUP BY u.courseid;
""")

results = cursor.fetchall()
print(f"Количество таких курсов: {len(results)}")
if results:
    print("ID курса - Кафедры")
    for row in results:
        print(f"{row[0]} - {row[1]}")

Количество таких курсов: 60
ID курса - Кафедры
71495 - ГМиТТК, ПиСЗ, УиИС
71508 - ПиСЗ, УиИС
71541 - ПиСЗ, УиИС
71547 - ПиСЗ, УиИС
71549 - ГМиТТК, ТиЭС
71571 - ЛиУТС, ПиСЗ, УиИС
71632 - CC, ВТиП
71736 - ГМДиОПИ, ЭиМЭ
71852 - АЭПиМ, ЭПП
71857 - АЭПиМ, ЭПП
71884 - АЭПиМ, ЭПП
71892 - АЭПиМ, ТиЭС, ЭПП
71904 - АЭПиМ, ЭПП
72126 - АЭПиМ, УиИС
72314 - Дизайна, ЛПиМ
72347 - Дизайна, ЛПиМ
72358 - МиХТ, ТОМ
72359 - ЛПиМ, МиХТ, ТОМ
72380 - ГМДиОПИ, ЛПиМ, МиХТ, ТОМ
72392 - ЛПиМ, МиХТ, ТОМ
72402 - ГМДиОПИ, ЛПиМ, МиХТ, ТОМ
72416 - ЛПиМ, МиХТ, ТОМ
72447 - ЛПиМ, МиХТ, ТОМ
72457 - ГМДиОПИ, ЛПиМ, МиХТ, ТОМ
72460 - ЛПиМ, МиХТ, ТОМ
72462 - МиХТ, ТОМ
72469 - МиХТ, ТОМ
72800 - АЭПиМ, ХОМ
72865 - Дизайна, ТОМ
72885 - Дизайна, ТОМ, ХОМ
73574 - CC, ДиСО
73986 - БИиИТ, Эконом.
75810 - ГМДиОПИ, ГМиТТК, РМПИ
75833 - ГМДиОПИ, ГМиТТК, ЛиУТС, РМПИ
75834 - ГМДиОПИ, ГМиТТК, РМПИ
75839 - ГМДиОПИ, ГМиТТК, РМПИ
75849 - ГМДиОПИ, ГМиТТК, РМПИ
76293 - ПиСЗ, Эконом.
78057 - АСУ, АЭПиМ, ВИ, ВТиП, ГМДиОПИ, ГМиТТК, ГМУиУП, ДиСО,

## Задание 5:

Выведите количество студентов, которые получили 2, 3, 4, 5.

Пример вывода:

| namer_level |	count |
|-----|------|
|2 |	4 |
|3 |	3435 |
|4 | 	4676765|
|5 | 232 |


In [ ]:

conn.rollback()

In [22]:
cursor.execute(''' 
    SELECT namer_level, COUNT(DISTINCT userid)
    FROM user_logs
    WHERE namer_level IN ('2', '3', '4', '5')
    GROUP BY namer_level
    ORDER BY namer_level;
''')
rows = cursor.fetchall()

print("| namer_level | count |")
print("|-------------|-------|")
for row in rows:
    print(f"| {row[0]}           | {row[1]}  |")

| namer_level | count |
|-------------|-------|
| 2           | 1069  |
| 3           | 1884  |
| 4           | 3243  |
| 5           | 3407  |


## Задание 6:

Выведите студента, который больше всех работает на портале (у него максимальное количество логов за вест период обучения).

In [24]:
cursor.execute(''' 
    SELECT userid
    FROM user_logs
    GROUP BY userid
    ORDER BY MAX(s_all) DESC
    LIMIT 1;

''')
row = cursor.fetchone()
print(f"Студент с максимальной активностью: id {row[0]}")

Студент с максимальной активностью: id 21606


## Задание 7:

Выведите по каждой недели среднее количество всех событий на портале.

In [30]:
cursor.execute(''' 
    SELECT num_week, AVG(s_all)
    FROM user_logs
    GROUP BY num_week
    ORDER BY num_week;
''')
rows = cursor.fetchall()

print("Неделя | Среднее количество событий")
for row in rows:
    print(f"{row[0]} | {row[1]:.3f}")

Неделя | Среднее количество событий
6 | 13.797
7 | 9.616
8 | 8.029
9 | 9.393
10 | 8.209
11 | 10.022
12 | 9.382
13 | 10.014
14 | 9.860
15 | 10.354
16 | 10.285
17 | 10.516
18 | 9.671
19 | 11.113
20 | 14.447
21 | 18.504
22 | 22.487
23 | 22.262
24 | 23.012
25 | 18.216
26 | 8.603
27 | 1.253
28 | 0.090
29 | 0.055


## Задание 8: 

Выведите название кафедры, у которой больше всего отличников.

Отдельно выведите название кафедры, у которой больше всего двоечников. 

In [38]:
conn.rollback()

In [39]:
# Больше всего отличников
cursor.execute("""
SELECT d.name, COUNT(DISTINCT u.userid) as student_count
FROM user_logs u
JOIN departments d ON u.depart = d.id
WHERE u.namer_level = '5'
GROUP BY d.name
ORDER BY student_count DESC
LIMIT 1;
""")
print(f"Больше всего отличников на кафедре: {cursor.fetchone()[0]}")
# Больше всего двоечников
cursor.execute("""
SELECT d.name, COUNT(DISTINCT u.userid) as student_count
FROM user_logs u
JOIN departments d ON u.depart = d.id
WHERE u.namer_level = '2'
GROUP BY d.name
ORDER BY student_count DESC
LIMIT 1;
""")
print(f"Больше всего двоечников на кафедре: {cursor.fetchone()[0]}")

Больше всего отличников на кафедре: ДиСО
Больше всего двоечников на кафедре: Эконом.


## Задание 9:
Провести анализ пиковой активности студентов перед экзаменом (с использованием (Common Table Expression — CTE), оператор with).

Вывести, на какой неделе семестра студенты проявляли наибольшую активность в курсе в целом, и как эта активность распределяется между студентами-бюджетниками и контрактниками.

Пример вывода :

| name_osno | week_number	| avg_s_all	| avg_s_course_viewed |	avg_s_q_attempt_viewed |
|-----|------|------|------|------|
| бюджет |	14	| 125.45 |	45.67 |	32.12 |
|контракт |	14	| 98.76 |	38.90 |	25.43 |

In [43]:
import pandas as pd
from IPython.display import display
cursor.execute(''' 
    WITH Activity AS (
        SELECT 
               CASE name_osno
                   WHEN '1' THEN 'бюджет' 
                   WHEN '2' THEN 'контракт' 
                   ELSE name_osno
               END as name_osno_text, 
               num_week, 
               AVG(s_all) as avg_s_all, 
               AVG(s_course_viewed) as avg_course_viewed, 
               AVG(s_q_attempt_viewed) as avg_q_attempt
        FROM user_logs
        GROUP BY name_osno, num_week
    )
    SELECT name_osno_text, num_week, avg_s_all, avg_course_viewed, avg_q_attempt
    FROM Activity
    ORDER BY avg_s_all DESC
    LIMIT 10;
''')
rows = cursor.fetchall()
columns = ["name_osno", "week_number", "avg_s_all", "avg_s_course_viewed", "avg_s_q_attempt_viewed"]
df = pd.DataFrame(rows, columns=columns)
df = df.sort_values(by="week_number", ascending=True).reset_index(drop=True)
display(df)

,name_osno,week_number,avg_s_all,avg_s_course_viewed,avg_s_q_attempt_viewed
0,бюджет,6,16.2509067461916660,5.3450471508019666,1.00120899492222132667
1,бюджет,21,19.1902152010961554,4.0288546788103490,2.8246151366164262
2,контракт,21,16.7533401849948613,3.0454265159301131,2.1749229188078109
3,контракт,22,23.5815005138746146,4.2441932168550874,4.0719424460431655
4,бюджет,22,22.0582735552510679,4.1630531151769163,4.5078584669944386
5,контракт,23,22.7886947584789311,4.1262076053442960,3.7714285714285714
6,бюджет,23,22.0548077698073668,4.0643991295236560,4.9590553719674377
7,контракт,24,28.6877697841726619,4.7344295991778006,5.4182939362795478
8,бюджет,24,20.7865720963971951,3.7824615136616426,4.9416458450874506
9,контракт,25,23.3646454265159301,4.2464542651593011,3.8752312435765673
